In [1]:
from hdrh.histogram import HdrHistogram
import seaborn as sns
import pandas
from matplotlib import pyplot as plt
import os.path
from enum import Enum
import matplotlib as mpl
from typing import *
import pandas as pd

In [2]:
RUNID = ''

RUNID1 = 'boar-2022-04-10-Sun-105859'
RUNID2 = 'boar-2022-04-12-Tue-110137'
RUNIDG1_2 = 'boar-2022-04-11-Mon-192342'
RUNID78_1 = 'boar-2022-04-10-Sun-144212'
RUNID78_2 = 'boar-2022-04-11-Mon-192342'
RUNID10X_1 = 'boar-2022-04-10-Sun-114225'
RUNID10X_2 = 'boar-2022-04-11-Mon-203414'
DACAPO = 'dacapochopin-29a657f'

DATA = {
    'G1 1.3x': [
        RUNID1 + '/lusearch.1319.70.jdk-lxr.g1.common.hs_pauses.dacapochopin-29a657f',
        RUNIDG1_2 + '/lusearch.1319.70.jdk-lxr.g1.common.hs_pauses.dacapochopin-29a657f',
    ],
    'Shen. 1.3x': [
        RUNID1 + '/lusearch.1319.70.jdk-lxr.shenandoah.common.hs_pauses.dacapochopin-29a657f',
        RUNID2 + '/lusearch.1319.70.jdk-lxr.shenandoah.common.hs_pauses.dacapochopin-29a657f',
    ],
    'Shen. 7/8 1.3x': [
        RUNID78_1 + '/lusearch.1319.70.jdk-lxr.shenandoah.common.hs_pauses.hs_cgc-28.dacapochopin-29a657f',
        RUNID78_2 + '/lusearch.1319.70.jdk-lxr.shenandoah.common.hs_pauses.hs_cgc-28.dacapochopin-29a657f',
    ],
    'LXR 1.3x': [
        RUNID1 + '/lusearch.1319.70.jdk-lxr.ix.common.tph.trace2-5.srv-128.srvw.lfb-32.dacapochopin-29a657f',
        RUNID2 + '/lusearch.1319.70.jdk-lxr.ix.common.tph.trace2-5.srv-128.srvw.lfb-32.dacapochopin-29a657f',
    ],
    'Shen. 10x': [
        RUNID10X_1 + '/lusearch.10000.530.jdk-lxr.shenandoah.common.hs_pauses.dacapochopin-29a657f',
        RUNID10X_2 + '/lusearch.10000.530.jdk-lxr.shenandoah.common.hs_pauses.dacapochopin-29a657f',
    ],
}

MAX_INVOCATIONS = 20

In [3]:
def load_data(invocation: int, folder: str):
    path = os.path.realpath(os.path.expanduser('{}.{}/pauses.csv'.format(folder, invocation)))
    if not os.path.isfile(path):
        return None
    df =  pandas.read_csv(path, names=["nanos"])
    df["millis"] = df["nanos"] / 1000000
    df["micros"] = df["nanos"] / 1000
    return df

def process_data(data: Optional[Dict[str, List[str]]] = None, invocations = MAX_INVOCATIONS):
    print(f'Loading...')
    # Clean up inputs
    if data is None:
        data = { k: v for k, v in DATA.items() }
    # Load data
    data2 = {}
    for gc, folders in data.items():
        data2[gc] = [
            f'~/MMTk-Dev/evaluation/results/log/{x}'.format(runid=RUNID, dacapo=DACAPO)
            for x in folders
        ]
    data = data2
    data: Dict[str, List[str]]
    # Process data
    processed_data = {'gc': [], 'p50': [], 'p99': [], 'p999': [], 'p9999': []}  
    for gc, folders in data.items():
        dfs = []
        num = 0
        for folder in folders:
            for i in range(invocations):
                x = load_data(i, folder)
                if x is not None:
                    num += 1
                    dfs.append(x)
        df = pd.concat(dfs, ignore_index=True)
        processed_data['gc'].append(gc)
        processed_data['p50'].append(df['millis'].quantile(0.5))
        processed_data['p99'].append(df['millis'].quantile(0.99))
        processed_data['p999'].append(df['millis'].quantile(0.999))
        processed_data['p9999'].append(df['millis'].quantile(0.9999))
        print(f'{gc}: {num} invocations')
    df = pd.DataFrame(processed_data)  
    df.to_csv('pauses-1.3x-punchline.csv', index=False)
    return df

In [4]:
process_data()

Loading...
G1 1.3x: 19 invocations
Shen. 1.3x: 20 invocations
Shen. 7/8 1.3x: 20 invocations
LXR 1.3x: 20 invocations
Shen. 10x: 20 invocations


,gc,p50,p99,p999,p9999
0,G1 1.3x,0.431060,0.863968,1.077166,1.224740
1,Shen. 1.3x,0.125837,0.349894,2.219795,3.017639
2,Shen. 7/8 1.3x,0.192121,0.583712,3.697111,4.179148
3,LXR 1.3x,0.864674,1.438700,1.786982,2.867127
4,Shen. 10x,0.226617,0.612150,0.754191,0.875450
